In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
from torch import bfloat16

In [2]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [3]:
model_id = "daryl149/llama-2-7b-chat-hf"
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    #use_auth_token=hf_auth
)

In [4]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    #quantization_config=bnb_config,
    device_map='auto',
    #use_auth_token=hf_auth
)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
model.eval()
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    #use_auth_token=hf_auth
)

In [ ]:
"### Instruction:"


In [7]:
stop_list = ['\nHuman:', '\n```\n']
device = 'cuda:1'

stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]


#stop_token_ids = [torch.LongTensor(x) for x in stop_token_ids]

from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [8]:
generate_text = transformers.pipeline(
    model=model, 
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    #stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [9]:
res = generate_text("Which team won the Serie A league in 2020?")
print(res[0]["generated_text"])

Which team won the Serie A league in 2020?

Answer: In 2020, the team that won the Serie A league was Juventus FC.
